In [ ]:
import pandas as pd
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Input, Activation, GRU, concatenate, Bidirectional 
from keras import Model
from keras import optimizers
from tensorflow import keras

In [ ]:
data_full = pd.read_csv('Daily Volatility Dataset.csv')

In [ ]:
X = data_full[["Volume", "Return", "Return_Squared", "EMAF", "Daily Volatility"]]
Y = data_full["target"]
data_set = data_full[["Volume", "Return", "Return_Squared", "EMAF", "Daily Volatility", "target"]]

In [ ]:
splitlimit = int(len(data_set)*0.8)
training_features, test_data = data_set[:splitlimit], data_set[splitlimit:]

In [ ]:
#Scaling the data
from sklearn.preprocessing import MinMaxScaler
scaler1 = MinMaxScaler()
scaler2 = MinMaxScaler()
training_data_features_scaled = scaler1.fit_transform(training_features[["Volume", "Return", "Return_Squared", "EMAF", "Daily Volatility"]])
dataset_scaled = scaler2.fit_transform(training_features)

In [ ]:
#Reconstructing training data for logistic regression
Z = []

backcandles = 10

for j in range(5):
    Z.append([])
    for i in range(backcandles, training_data_features_scaled.shape[0]):
        Z[j].append(training_data_features_scaled[i-backcandles:i, j])
        
Z = np.moveaxis(Z, [0], [2])        
Z, yi = np.array(Z), np.array(dataset_scaled[backcandles-1:, -1])
y_final = np.reshape(yi,(len(yi),1))
y_final = y_final[:-1]

In [ ]:
test_data_features_scaled = scaler1.transform(test_data[["Volume", "Return", "Return_Squared", "EMAF", "Daily Volatility"]])
test_dataset_scaled = scaler2.transform(test_data)

In [ ]:
#Reconstructing test data for logistic regression
T = []

backcandles = 10

for j in range(5):
    T.append([])
    for i in range(backcandles, test_data_features_scaled.shape[0]):
        T[j].append(test_data_features_scaled[i-backcandles:i, j])
        
        
T = np.moveaxis(T, [0], [2])   
T, yi_test = np.array(T), np.array(test_dataset_scaled[backcandles-1:, -1])
y_final_test = np.reshape(yi_test,(len(yi_test),1))
y_final_test = y_final_test[:-1]

In [ ]:
#Logistic Regression model
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from matplotlib import cm
import numpy as np

Z_flattened = Z.reshape(3807, -1)
y_final_flattened = y_final.ravel()

T_flattened = T.reshape(945, -1)
y_final_test_flattened = y_final_test.ravel()


model_LR = LogisticRegression()
model_LR.fit(Z_flattened, y_final_flattened)

LR_predictions = model_LR.predict(T_flattened)

y_frame_LR = test_data[["target"]].tail(955- backcandles +1)
y_frame_LR = y_frame_LR.iloc[:-1]
y_frame_LR['predicted'] = LR_predictions

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_frame_LR['predicted'], y_frame_LR['target'])
print(cm)

fpr, tpr, thresholds = roc_curve(y_frame_LR['target'], y_frame_LR['predicted'])
roc_auc = auc(fpr, tpr)

roc_auc

In [ ]:
#FFNN model
input_shape = len(training_data_features_scaled[0, :]) 

ff_input = Input(shape=input_shape, name='ff_input')

inputs = Dense(80, activation='relu', name='first_layer')(ff_input)  

# Output layer
outputs = Dense(1, activation='sigmoid', name='output')(inputs)

# Create the model
model = Model(inputs=ff_input, outputs=outputs)

# Compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

history = model.fit(x=training_data_features_scaled, y=dataset_scaled[:, -1], epochs=50, validation_data=(training_data_features_scaled, dataset_scaled[:, -1]))

In [ ]:
FFNN_predictions = model.predict(test_data_features_scaled)
FFNN_predictions_classes = (FFNN_predictions > 0.5).astype(int)
y_frame_FFNN = test_dataset_scaled[:, -1]

df = pd.DataFrame({
    "target": y_frame_FFNN,
    "predicted": FFNN_predictions_classes.ravel()  
})

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(df['predicted'], df['target'])
print(cm)

In [ ]:
fpr, tpr, thresholds = roc_curve(df['target'], df['predicted'])
roc_auc = auc(fpr, tpr)
roc_auc